<h1>Kapitel 7: Funktionen (Teil 2): Rekursion, Namensräume, Scopes</h1>
<p>Grundlagen der Programmierung 1<p><h2>Holger Karl</h2>
 

<h1>Overview</h1> 

1. Überblick
1. Flexible Funktionsparameter und Rückgabewerte
1. Funktionen rufen Funktionen auf
1. Indirekte Rekursion
1. Direkte Rekursion
1. Funktionen und Software-Entwurf
1. Zusammenfassung 
 


<h1>Overview</h1> 

1. **<font color="red">Überblick</font>**
1. Flexible Funktionsparameter und Rückgabewerte
1. Funktionen rufen Funktionen auf
1. Indirekte Rekursion
1. Direkte Rekursion
1. Funktionen und Software-Entwurf
1. Zusammenfassung

# Setup

Main point here is to set up warnings properly for presentation and to
load tutormagic, so that we can later on use pythontutor for code
animations. 

In [ ]:
%load_ext tutormagic
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

import requests
import webbrowser
from IPython.core.magic import register_line_magic

pingo_token = "BRx9QJP7rjdUGBq3x4hP"

# Test-Session: 
pingo_url = "https://pingo.upb.de/events/311055/quick_start.json"
# Produktiv-Session: 
# pingo_url = "https://pingo.upb.de/events/204051/quick_start.json"

pingo_duration = '120'
pingo_type = "single"


@register_line_magic
def pingo(line):
    params = {'survey_name': pingo_title, 
              'predef_options': pingo_questions, 
              'q_type': pingo_type,
              'duration': pingo_duration, 
              'auth_token': pingo_token}

    r = requests.post(pingo_url, json=params)
    webbrowser.open_new_tab("https://pingo.upb.de" + r.headers['Location'])

# Was bisher geschah

Wir haben: 

-   Ausdrücke und Anweisungen
-   Namen verweisen auf Werte
-   Kontrollstrukturen für Ablauf: `if`, `while`, `for`
-   Funktionen als Zusammenfassung von Anweisungen
    -   Definition und Aufruf

# Dieses Kapitel

Wir kehren noch einmal zu Funktionen zurück und betrachten: 

-   Aufruf durch flexible Parameter
-   Funktionen rufen andere Funktionen (oder sich selbst) auf
-   Funktionsdefinition innerhalb anderer Funktionen
-   Die Regeln für Namensräume und  *Scopes*

<h1>Overview</h1> 

1. Überblick
1. **<font color="red">Flexible Funktionsparameter und Rückgabewerte</font>**
1. Funktionen rufen Funktionen auf
1. Indirekte Rekursion
1. Direkte Rekursion
1. Funktionen und Software-Entwurf
1. Zusammenfassung

# Erinnerung: Parameter von `print`

An `print` können Parameter flexibler übergeben werden als an selbst
definierte Funktionen: 

-   Beliebige Anzahl
-   Parameter können mit Namen versehen werden

Wäre das nicht auch für eigene Funktionen praktisch? 

# Erinnerung: Welche formale Parameter bekommen welchen Wert?

Mehrere formale Parameter in Funktionsdefinition: 

-   Argumente und formale Parameter werden von links nach rechts
    zugeordnet: a wird zu Namen für Wert von x, b für y, c für z
-   Sogenannte *Positionsparameter* (*positional arguments*)

In [ ]:
def f(a, b, c):
    # ... 

f(x, y, z)

# Variable Anzahl an Positionsparametern?

-   Bisherige Syntax von `def`: Feste Anzahl Parameter vorgegeben
-   Erweiterte Syntax: Beliebige Parameteranzahl erlauben
    -   Wie Funktionsrumpf die Werte zur Verfügung stellen?
        -   Als Aufzählung!
    -   In Funktionsdefinition: **einem** Parameter einen Stern 
        voranstellen
        -   Häufig: `*args` als formaler Parametername (nur Konvention!)
    -   Nach Stern-Parameter darf kein normaler Parameter folgen

# Variable Anzahl an Positionsparametern &#x2013; Beispiel

In [ ]:
def f(a, b, *args):
    # a und b sind normale Parameter
    # args ist eine Aufzählung mit ggf. weiteren Parameter
    print("a:", a, "b:", b)
    print("Anzahl weitere Parameter:", len(args))
    print("weitere Parameter:", args)
    print("-------------")

# f mit zwei Parametern aufrufen:     
f(1, 2)
# f mit zusätzlichen, optionalen Parametern aufrufen:
f(3, 4, "a")
f(5, 6, "b", 17, "c")
# das würde scheitern: 
# f(1)

# Formale Parameter mit vorbelegten Werten (*default*)

-   Möglicherweise haben manche formale Parameter häufig vorkommende,
    typische Werte (*defaults*)
-   Die der Funktionsentwickler kennt
-   Und die der Aufrufer nicht jedes Mal neu hinschreiben will
-   Syntax für defaults: Im Funktionskopf default mit `=` notieren
    -   Müssen bei Aufruf nicht vorhanden sein

In [ ]:
def f(a, b=42):
    """Parameter b hat default-Wert 42, wenn nicht
    explizit anders übergeben."""
    print("a:", a, end=", ")
    print("b:", b)

# Aufruf ohne default-Parameter
f(1)
# Parameter nach Reihenfolge
f(2, 3)

# Argument bei Aufruf benennen

-   Formale Parameter haben Namen
-   Möglicherweise bei Aufruf angenehmer, die Namen zu nennen, statt die
    Position zu wissen?
-   Sogenannte Schlüssel/Wert-Paare (***keyword arguments***)
    -   Überschreiben ggf. vorhandene default-Werte
-   Syntax: Als Zuweisung bei Aufruf
-   Beispiel:

In [ ]:
def f(a, b=42):
    print("a:", a, end=", ")
    print("b:", b)

# Aufruf nach Position:
f(1, 2)
# Aufruf mit Nennung der Parameter:
f(a=3, b=4)
# Bei Nennung der formalen Parameter Reihenfolge unwichtig:
f(b=5, a=6)

# `dict` als Schlüssel-/Wert-Paare übergeben

-   Keyword arguments: Abbildung von Namen auf Wert
-   Das kennen wir von `dict`!
    -   Kann man ein `dict` als Folge Schlüssel-Wert-Paaren an Funktion
        übergeben?
-   Syntax: Ein dict mit **zwei** vorangestellten Sternen wird als Folge
    von entsprechenden Argumenten aufgefasst

In [ ]:
def f(a, b, c):
    print("a:", a, end=", ")
    print("b:", b, end=", ")
    print("c:", c)

d = {'a': 17, 'b': 42, 'c': 99}
# Kann das funktionieren? 
# f(d)
# Besondere Aufrufssyntax: 
f(**d)

# `dict` als Schlüssel-/Wert-Paare übergeben &#x2013; Fehlende Schlüssel

Was passiert hier, was muss man ändern? 

In [ ]:
def f(a, b, c):
    print("a:", a, end=", ")
    print("b:", b, end=", ")
    print("c:", c)

d = {'a': 17, 'b': 42}
# Besondere Aufrufssyntax: 
f(**d)

# `dict` als Schlüssel-/Wert-Paare übergeben &#x2013; Zu viele Schlüssel?

In [ ]:
def f(a, b, c):
    print("a:", a, end=", ")
    print("b:", b, end=", ")
    print("c:", c)

d = {'a': 17, 'b': 42, 'c': 99, 'd': 1}

# Das scheitert: 
f(**d)

# `dict` als Schlüssel-/Wert-Paare übergeben &#x2013; Optionale Schlüssel?

-   Wir brauchen eine Vorkehrung, um beliebige Schlüssel/Wert-Paare
    aufzunehmen
    -   Ähnlich wie bei beliebigen Positionsparametern mit `*args`
-   Syntax entsprechend angelehnt:
    -   In Funktionskopf, einen Parameter mit **zwei** Sternen versehen
        -   Konvention: oft `**kwargs` (für keyword arguments)
    -   Alle überzähligen Schlüssel-/Wert-Paare werden in diesem Parameter
        als `dict` abgelegt

In [ ]:
def f(a, **kwargs):
    print("a:", a, end=", ")
    print("kwargs:", kwargs)

d = {'a': 17, 'b': 42, 'c': 99, 'd': 1}
# Besondere Aufrufssyntax: 
f(**d)

# Beispiel

In [ ]:
def f(a, b, *args, **kwargs):
    if 'x' in kwargs:
        return args[1], kwargs['y']
    else:
        return str(len(args)) * kwargs['z'] 

print(f("hallo", 3, 17, "gp1", **{'y': 17, 'z': 3}))

## Pingo!<a id="orgheadline21"></a>

In [ ]:
pingo_title = "Was gibt obiger Code aus?" 
pingo_questions = ["hallo gp1", "222",  '("gp1", 17)', "1717"]
pingo_duration = "180"

%pingo

# Zusammenfassung formale Parameter und Argumente &#x2013; Syntax

<table border="2" cellspacing="0" cellpadding="6" rules="groups" frame="hsides">


<colgroup>
<col  class="org-left" />

<col  class="org-left" />
</colgroup>
<thead>
<tr>
<th scope="col" class="org-left">Definitions-Syntax</th>
<th scope="col" class="org-left">Interpretation</th>
</tr>
</thead>

<tbody>
<tr>
<td class="org-left">`def f(a, b)`</td>
<td class="org-left">Positionsparameter</td>
</tr>


<tr>
<td class="org-left">`def f(a=17)`</td>
<td class="org-left">Parameter mit default</td>
</tr>


<tr>
<td class="org-left">`def f(*args)`</td>
<td class="org-left">Parameter für überzählige Positions-Argumente (als Tuple)</td>
</tr>


<tr>
<td class="org-left">`def f(**kwargs)`</td>
<td class="org-left">Parameter für überzählige benannte Argumente (als Dict)</td>
</tr>
</tbody>

<tbody>
<tr>
<td class="org-left">Aufruf-Syntax</td>
<td class="org-left">Interpretation</td>
</tr>
</tbody>

<tbody>
<tr>
<td class="org-left">`f(x, y)`</td>
<td class="org-left">Positionsargument</td>
</tr>


<tr>
<td class="org-left">`f(a=5)`</td>
<td class="org-left">Argument explizit benennen</td>
</tr>


<tr>
<td class="org-left">`f(*l)`</td>
<td class="org-left">Siehe Übungsblatt</td>
</tr>


<tr>
<td class="org-left">`f(**d)`</td>
<td class="org-left">Dict d ist Liste der benannten Argumente</td>
</tr>
</tbody>
</table>

Bemerkung: Python 3 hat noch ein paar weitere Feinheiten; brauchen wir
zunächst nicht 

# Tuple als Rückgabewerte

Eigentlich selbstverständlich: Tuple sind legale Rückgabewerte 

-   Damit einfache Möglichkeit, aus Funktion mehr als einen Wert
    zurückzugeben
-   `return` Tuple
-   Automatisches unpacking bei Funktionsaufruf

In [ ]:
def mehrfache_rueckgabe():
    x = 10
    y = [1, 2, 3]
    return x, y

# Tuple entgegennehmen:
t = mehrfache_rueckgabe()
print("t:", t)
# Unpacking des tuples: 
a, b = mehrfache_rueckgabe()
print("a:", a, "b:", b)

<h1>Overview</h1> 

1. Überblick
1. Flexible Funktionsparameter und Rückgabewerte
1. **<font color="red">Funktionen rufen Funktionen auf</font>**
1. Indirekte Rekursion
1. Direkte Rekursion
1. Funktionen und Software-Entwurf
1. Zusammenfassung

# Funktionen rufen Funktionen auf

-   Bis jetzt: lediglich eine Funktion, die aufgerufen wurde
    -   Stimmt nicht ganz: Unsere Funktion hat auch schon die Funktion
        `print` aufgerufen
-   Was passiert genau, wenn eine Funktion eine andere aufruft?
    -   Insbesondere: Welche Namen sind sichtbar?

# Ausführungsmodell

Das Ausführungsmodell ist ganz naheliegend

-   Aufrufende Funktion unterbricht Ablauf bei Aufruf
-   Aufgerufene Funktion wird ausgeführt
-   Rückgabewert nimmt den Platz des Aufrufs in der aufrufenden
    Funktion ein
-   Aufrufer macht weiter

# Illustration

In [ ]:
%%tutor 
def f():
    print("in f, vor g")
    x = g()
    print("in f, nach g", "x: ", x)

def g():
    y = 42
    return y

f()

# Beispiel: Matrix ausgeben

Beispiel: Zwei Funktionen, um eine Matix auszugeben 

In [ ]:
# eine verschachtelte Liste: 
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
# matrix wollen wir als eine 4x3 Matrix ausgeben 

def print_matrix(m):
    for zeile in m:
        print_zeile(zeile)
        # Neue Zeile
        print()

def print_zeile(z):
    for c in z:
        print(c, end=" ")

print_matrix(matrix)

# Beobachtung: Reihenfolge Definition und Aufruf

-   Im Programmcode steht ein Aufruf von `print_zeile` bevor
    `print_zeile` definiert wurde
-   Hier offenbar unproblematisch
    -   Weil `def print_matrix` eine Anweisung ist, die lediglich den
        Namen der Funktion mit dem Programmcode verbindet
    -   Dabei wird noch nichts ausgeführt!
-   Entscheidend ist der Zeitpunkt der Ausführung des Aufrufs!
    -   Dann muss der Funktionsname `def`-iniert worden sein

# Beobachtung: Reihenfolge Definition und Aufruf

Funktionsname vor Aufruf definiert &#x2013; das hier scheitert: 

-   Fehlermeldung: `NameError: name 'f' is not defined`

In [ ]:
f()

def f():
    print("Hallo")

# Namen und Sichtbarkeit?

Auf welche Namen kann man in einer Funktion zugreifen? 

-   Natürlich auf die selbst zugewiesenen
-   Andere?

# Sichtbarkeit von Namen außerhalb der Funktion?

## Beispiel 1: Name vor Funktionsdefinition zugewiesen<a id="orgheadline32"></a>

In [ ]:
%%tutor -t
x = 5
def f():
    y = x 
    print(y)
f()

## Beispiel 2: Name **nach** Funktionsdefinition zugewiesen<a id="orgheadline33"></a>

In [ ]:
def f():
    y = x 
    print(y)
x = 5 
f()

# Offenbar sichtbar!

-   Bei Zugriff auf `x` stellt `f` fest, dass es kein `x` im eigenen
    Scope gibt
-   Dann beginnt Suche: gibt es ein `x` im Scope des aufrufenden
    Programmteils?
    -   Ja!
    -   Dann benutzen wird doch das `x`!
-   Diese Suche findet während der **Ausführung** von `f` statt, nicht
    während der **Definition** von `f`
    -   Deswegen ist es hier egal, ob `x` vor oder nach def Definition von
        `f` belegt wird

# Offenbar sichtbar! &#x2013; Beispiel

Was wird hier ausgegeben? 

In [ ]:
x = 3
def f():
   y = x
   print(y)
x = 5
f()
x = 7

## Pingo<a id="orgheadline36"></a>

In [ ]:
pingo_title = "Ausgabe?" 
pingo_questions = ["3", "5", "7", "42"]

%pingo

# Sichtbarkeit bei schreibendem Zugriff?

Funktioniert das hier?  

-   Nur minimale Änderung zu oben!

In [ ]:
x = 5
def f():
    x = x + 1
    print(x)
f()

# Scope durch Zuweisung

-   **Zuweisung** an Name bestimmt ihren Scope

# Sichtbarkeit von Namen **zwischen** zwei Funktionen?

-   Aufruf von `g` durch `f` oben hatte keine Namen
-   Was passiert, wenn wir das hinzufügen?

In [ ]:
def f():
    x = 17
    g()

def g():
    z = a
    z += x
    print(z)

a = 42
f()

## Fehlermeldung<a id="orgheadline40"></a>

-   In Funktion `g`:  `NameError: global name 'x' is not defined`

# Unterschied der beiden Fälle?

Offenbar unterschiedliches Verhalten 

-   Namen innerhalb oder außerhalb einer Funktion definiert
-   Namen außerhalb *aller* Funktionen existieren in einem besonderem Scope:
    dem *global scope*
-   Namen innerhalb einer Funktion existieren nur im Scope der Funktion
    -   Der *local scope* der Funktion
    -   Local scopes zweier Funktionen haben nichts miteinander zu tun!
        -   Genauer: Die Scopes der Namensräume der Funktionen&#x2026;
        -   (Scheinbar) gleicher Name in unterschiedlichen Scopes:
            Unterschiedlicher Namer!

# Disjunkte local scopes &#x2013; Beispiel

In [ ]:
%%tutor -t
x = 99

def f():
    x = 17
    print("x in f:", x)
    g()
    print("x in f:", x)

def g():
    x = 42
    print("x in g:", x)

f()

# Disjunkte local scopes &#x2013; Beispiel 2

-   Fast das gleiche Beispiel wie eben
-   Allerdings wird in `g` nicht an `x` zugewiesen
-   Was gibt `g` aus? Erklärung?

In [ ]:
%%tutor -t
x = 99

def f():
    x = 17
    print("x in f:", x)
    g()
    print("x in f:", x)

def g():
    print("x in g:", x)

f()

# Modifizierende Nutzung globaler Namen

-   Zuweisung erzeugt einen Namen im lokalen Scope
    -   Genauer: Im Namensraum des lokalen Scopes
-   Was passiert, wenn an  einen globale Namen zugewiesen wird?

## Zuweisung erzeugt lokalen Namen<a id="orgheadline45"></a>

Wie oben &#x2013;  das hier funktioniert nicht: 

In [ ]:
x = 5
def f():
    x = x + 1

f()

# Modifikation eines globalen  Namens

Das hier funktioniert: 

In [ ]:
L = [1, 2]
def f():
    L.append(3)

f()
print(L)

## Beobachtung<a id="orgheadline47"></a>

-   An `L` wird nicht zugewiesen, also wird das globale `L` benutzt
-   Es wird nicht der Name `L` verändert, sondern das referenzierte
    Objekt, die Liste selbst!

# Modifikation einer globalen Variablen

-   Eigentlich geht das nicht!
    -   Funktionen kommunizieren Ergebnisse durch
        -   Rückgabewerte (gut!)
        -   Ggf. durch Veränderung globaler Objekte (zweischneidig!)
-   Aber wenn doch nötig?

<center>![img](./figures/DangerWillRobinson.png "Danger, danger!")</center>

# Modifikation eines globalen Namens

-   Eigenes Schlüsselwort: `global`
    -   Deklariert einen Namen als dem globalen Scope zugehörig
-   **Extrem bedenklich**!!
    -   Schwer lesbar, schlecht wartbar, unübersichtlich, fehleranfällig,
        &#x2026;

In [ ]:
x = 5
def f():
    global x
    x = x + 1

f()
print(x)

# Die Gefahr globaler Namen

-   Die Modifikation globaler Namen wird durch den  Sprachentwurf bewusst
    **erschwert**
-   Kernproblem: Programm ist nicht mehr verständlich, indem man nur den
    jeweiligen Ausschnitt betrachtet

## Beispiel: Welchen Wert hat `X`?<a id="orgheadline51"></a>

In [ ]:
X = 99
def func1():
    global X
    X = 88

def func2():
    global X
    X = 77

func2()
print(X)

## Pingo<a id="orgheadline52"></a>

In [ ]:
pingo_title = "Welcher Wert wird ausgegeben?" 
pingo_questions = ["77", "88", "99"]

%pingo

# Zusammenfassung: Scope von Namen  in Funktionen

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: Scope von Namen<a id="orgheadline54"></a></b></div>

Der Scope eines Namen bestimmt sich daraus, wo an diesen Namen 
eine **Zuweisung** erfolgt. 

-   Namen, an die innerhalb eines `def` zugewiesen wurde, sind im
    local scope dieser Funktion
-   Namen, an die außerhalb aller Funktionen zugewiesen wurde, sind
    im global scope
    -   Verweist ein Name im global scope auf ein Objekt, so kann das
        Objekt durchaus verändert werden!
-   Auf Namen im global scope kann von einer Funktion aus nur lesend
    zugegriffen werden 
    -   Es sei denn, ein solcher Name ist vorher als  `global` deklariert

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

<h1>Overview</h1> 

1. Überblick
1. Flexible Funktionsparameter und Rückgabewerte
1. Funktionen rufen Funktionen auf
1. **<font color="red">Indirekte Rekursion</font>**
1. Direkte Rekursion
1. Funktionen und Software-Entwurf
1. Zusammenfassung

# Funktionen rufen sich gegenseitig auf

-   Bisher: `f` hat `g` aufgerufen.
    -   Und würde `g` eine Funktion `h` aufrufen, so ginge das genauso
-   Was passiert, wenn `g` dann wieder `f` aufruft?

Zwei Fragen: 

-   Terminierung?
-   Scope?

# Wechselseitiger Aufruf: (Inkorrektes) Beispiel

In [ ]:
def f():
    print("f")
    g()

def g()
    print("g")
    f()

f()

## Problem<a id="orgheadline57"></a>

Endlos, terminiert nicht!

# Wechselseitiger Aufruf: Terminierung

-   Ziel: Stelle sicher, dass die Folge von Aufrufen terminiert
-   Idee: Ähnlich wie Terminierung von Schleifen 
    -   Finde einen Ausdruck, der mit jedem Ausdruck streng monoton und
        diskret fällt
    -   Zeige, dass Aufrufe enden, wenn Ausdruck einen Wert erreicht
        (z.B. 0)

# Wechselseitiger Aufruf: Beispiel

Was tut dieses Programm? 

In [ ]:
def is_even(x):
    print("is_even checks:", x)
    if x == 0:
        return True
    else:
        y = x - 1
        return is_odd(y)

def is_odd(x):
    print("is_odd checks:", x)
    if x == 0:
        return False
    else:
        y = x - 1
        return is_even(y)

result = is_even(5)
print(result)

# Wechselseitiger Aufruf: Scopes und Namensräume

Schauen wir uns die Scopes bzw. Namensräume genauer an: 

-   Beide Funktionen `is_even` und `is_odd` haben Namen `x` und `y`
-   Diese `x` und `y` sind in unterschiedlichen Scopes, also kein
    Problem
-   Aber was passiert mit einem `x` von `is_even`, wenn `is_even` erneut
    aufgerufen wird? Gleicher Scope???
-   Was würden Sie sinnvollerweise erwarten?

# Wechselseitiger Aufruf: Animation

In [ ]:
%%tutor -t
def is_even(x):
    print("is_even checks:", x)
    if x == 0:
        return True
    else:
        y = x - 1
        return is_odd(y)

def is_odd(x):
    print("is_odd checks:", x)
    if x == 0:
        return False
    else:
        y = x - 1
        return is_even(y)

result = is_even(5)
print(result)

# Zugriff auf Namen in anderem Namensraum?

-   Im oben Beispiel mit `f` und `g` gelang es nicht, auf eine Variable
    `x` in einem anderen Namensraum zuzugreifen
-   Geht das hier?
    -   Kann eine Funktion `is_even` auf eine Variable `x` *von sich
        selbst* zugreifen? In einem anderen Namensraum von `is_even`
        (allerdings im gleichen Scope)?
    -   **Nein!**
        -   Man kann das nicht einmal hinschreiben
        -   Welches `x` würde man auch meinen?

# Rekursion und Namensraum

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: Jeder Funktionsaufruf hat einen eigenen Namensraum<a id="orgheadline64"></a></b></div>

Jeder rekursive **Funktionsaufruf** hat einen eigenen Namensraum. Die
Namen in einem Namensraum haben nichts mit Namen in einem
anderen Namensraum zu tun, selbst wenn es sich um den gleichen Scope
handeln sollte. 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

<h1>Overview</h1> 

1. Überblick
1. Flexible Funktionsparameter und Rückgabewerte
1. Funktionen rufen Funktionen auf
1. Indirekte Rekursion
1. **<font color="red">Direkte Rekursion</font>**
1. Funktionen und Software-Entwurf
1. Zusammenfassung

# Direkte Rekursion

-   Sonderfall der Rekursion: Eine Funktion ruft sich selbst auf 
    -   Terminierung sicherstellen!
-   Allgemeiner: *Definition nimmt auf sich selbst Bezug*

# Rekursion in der Kunst

Abbildung [152](#orgparagraph1) illustriert Rekursion. 

<center>![img](./figures/bild-rekursion.png "Bild im Bild")</center>

# Rekursion in der Kunst

Abbildung [154](#orgparagraph2) illustriert Rekursion. 

<center>![img](./figures/escher.png "Rekursion: Zeichnende Hände")</center>

# Beispiel: Fakultät

## Definition, nicht-rekursiv<a id="orgheadline69"></a>

Für $n\geq 1$: 

$\mathrm{fak}(n) = \Pi _{i=1}^n i = 1 \cdot \ldots \cdot n$ 

## Definition, rekursiv<a id="orgheadline70"></a>

$$\mathrm{fak}(n) = \begin{cases}
1 & n = 0 \\ 
1 & n = 1 \\
n \cdot \mathrm{fak}(n-1) & \mathrm{sonst.}  
\end{cases}$$ 

# Beispiel: Fakultät &#x2013; Code

In [ ]:
%%tutor -t
def fak(n):
    if n==1:
        return 1
    else:
        return n*fak(n-1)

res = fak(5)
print(res)

# Randfälle?

Was passiert hier? Notwendige Änderungen? 

In [ ]:
fak(0)
fak(-5)

# Fakultät: Scope und Namensraum

Um das nochmals zu betonen: 

-   Der *Namensraum* eines  lokalen Namens  `n` in einer Funktion
    `fak`  ist bei jedem Aufruf von `fak` ein neuer; jeder
    Aufruf hat einen eigenen Namensraum
-   Der *Scope* des Namens `n` ist bei allen Aufrufen von `fak` der
    gleiche: die Zeilen 2&#x2013;6

# Beispiel: String umdrehen

-   Gegeben: Zeichenkette
-   Gesucht: Die Zeichenkette, rückwärts gelesen

## Rekursive Idee<a id="orgheadline75"></a>

-   Die einfachen Fälle: 
    -   Eine leere Zeichenkette rückwärts: die leere Zeichenkette
    -   Eine Zeichenkette der Länge 1 rückwärts: die Zeichenkette selbst
-   Die interessanten Fälle: eine längere Kette
    -   Nimm das erste Zeichen von der Kette weg
    -   Drehe die verkürzte Kette um
    -   Hänge das erste Zeichen ans Ende der umkehrten, verkürzten Kette
        an

# Beispiel: String umdrehen &#x2013; Code

In [ ]:
def reverse(s):
    if len(s)<=1:
        return s
    else:
        first = s[0]
        rest = reverse(s[1:])
        return rest + first

print(reverse("Hallo GP1"))

# Rekursion &#x2013; Muster

-   Rekursion beruht auf
    -   der Berechnung einfacherer Teilprobleme
    -   Zusammensetzen der Lösung dieser Teilprobleme zur Gesamtlösung
    -   Ganz einfache Teilprobleme werden direkt gelöst
-   Typisches Muster:

In [ ]:
def rekursive_Funktion(parameter):
    if (parameter einfach genug für direkte Loesung):
        return Funktionswert beruhend auf parameter
    else:
        Bestimme einfacherere(s) Teilproblem(e)
        Rufe rekursive_Funktion fuer Teilprobleme auf
        Berechne Funktionswert beruhend auf Loesungen der Teilprobleme

# Rekursion &#x2013; Entwurf?

Wie entwirft man rekursive Algorithmen? 

-   Suche einfache **Grenzfälle**: Für welche Eingaben kann man das Ergebnis
    direkt angeben?
-   Suche **Zerteilung**: Wie kann man kompliziertere Eingaben in einfachere
    Eingaben zerteilen, komplexe Probleme in einfachere Teilprobleme
    zerlegen?
-   Suche **Zusammensetzung**: Wie kann man Lösungen der Teilprobleme zur
    Lösung des Gesamtproblems zusammensetzen?

# Entwurfsbeispiel: Türme von Hanoi

-   Auf einem Turm liegt ein Stapel Scheiben unterschiedlicher Größe
    -   Kleinere Scheiben müssen auf größeren Scheiben liegen
-   Dieser Scheibenstapel soll auf einen anderen Turm transportiert
    werden
-   Es kann immer nur eine Scheibe bewegt werden
    -   Kleinere nur auf größeren, nicht umgekehrt!
-   Es gibt einen dritten Turm, der als Zwischenablage verwendet werden
    kann

# Türme von Hanoi &#x2013; Start

<center>![img](figures/hanoi.png "Türme von Hanoi: Start")</center>

# Türme von Hanoi &#x2013; Ziel

<center>![img](figures/hanoi2.png "Türme von Hanoi: Ziel")</center>

# Türme von Hanoi &#x2013; Möglicher Zwischenstand

<center>![img](figures/hanoi3.png "Türme von Hanoi: Zwischenstand")</center>

# Türme von Hanoi &#x2013; Fragen

Also: 

-   Grenzfall?
-   Zerteilung?
-   Zusammensetzung?

# Türme von Hanoi &#x2013; Code

In [ ]:
def hanoi(hoehe, von, nach, ablage):
    if hoehe==1:
        print("Lege Scheibe von", von, "nach", nach)
    else:
        hanoi(hoehe-1, von, ablage, nach)
        print("Lege Scheibe von", von, "nach", nach)
        hanoi(hoehe-1, ablage, nach, von)

hanoi(3, "start", "ziel", "ablage")

# Wie oft wird eine Scheibe bewegt?

-   Wie könnten wir das mitzählen? 
    -   Ordentliche Lösung?
    -   Unordentlich?

# Rekursion oder Iteration?

-   Fakultät lässt sich natürlich auch trivial iterativ (ohne rekursive
    Funktionsaufrufe) implementieren
-   Häufig:
    -   Rekursive Variante einfacher, übersichtlicher
    -   Iterative Variante schneller, komplizierter
-   Allgemein: Rekursiv und iterativ ineinander überführbar
-   Wann nimmt man was?
    -   Je nach Problem, Struktur, Relevanz der Lösungszeit
    -   Vgl. VL Datenstrukturen und Algorithmen

# Rekursion zur Definition

-   Grundidee der Rekursion auch für Definitionen anwendbar
    -   Für Funktionen, wie oben
    -   Für Datenstrukturen: Bäume , Listen
    -   Für Programmiersprachen:
        -   Eine Anweisung besteht aus&#x2026;
        -   Ein Ausdruck besteht aus&#x2026; anderen Ausdrücken!
-   Mehr dazu in VL Modellierung, VL DuA, und im ganzen Rest des
    Studiums!

# Rekursion: Realisierung?

-   Jeder Aufruf einer Funktion &#x2013; sei es normal, durch indirekte oder
    direkte Rekursion &#x2013; hat also einen eigenen Namensraum, eigene Version
    des Zustands der Funktion
    -   Zustand: Werte der Parameter, der lokalen Namen, des Rückgabewertes
-   Konzeptionell schön, aber wie wird das realisiert?
-   Kernidee: Ein *Stapel* von *Frames*
    -   Datenstruktur, auf der andere Daten abgelegt werden können
    -   Kann wachsen und schrumpfen
    -   Stellt Platz für die Namensräume bereit
    -   Details: VL GPS

<h1>Overview</h1> 

1. Überblick
1. Flexible Funktionsparameter und Rückgabewerte
1. Funktionen rufen Funktionen auf
1. Indirekte Rekursion
1. Direkte Rekursion
1. **<font color="red">Funktionen und Software-Entwurf</font>**
1. Zusammenfassung

# Wie nutzt man Funktionen sinnvoll?

-   Bis jetzt: Syntax und Semantik für Funktionen
    -   (Noch nicht vollständig)
-   Aber wie entwirft man Programme sinnvoll?
    -   Regel für die Nutzung von Funktionen?
-   Hier: einige erste Faustregeln
    -   Später: VL Software-Entwurf

# Funktionen und Coupling

## Eingabe mit Argumenten, Ausgabe mit return<a id="orgheadline91"></a>

-   Funktionen sollten Eingaben nur durch die Argumente erhalten
-   Resultate nur durch `return`
-   Unabhängig von Werten um die Funktion herum
-   *Isolation* der Funktion erhöht Wiederverwendbarkeit

## Globale Namen nur wenn nicht anders möglich<a id="orgheadline92"></a>

-   Globale Namen erzeugen Abhängigkeiten und Komplexität
-   Nur als letzter Ausweg! In 99.9999% **nicht** nötig

# Funktionen und Coupling (2)

## Veränderbare Argumente nicht verändern<a id="orgheadline94"></a>

-   Bekommt eine Funktion veränderbare Typen (Listen, Mengen, Dicts) als
    Argumente, so sollte sie  diese typischerweise nicht verändern
-   Nur, wenn dies der Aufrufer offensichtlich erwarten muss
-   Klar dokumentieren, wenn dies passiert!
-   Meist besser: Veränderte Werte mit `return` zurückgeben
    -   Aufrufer entscheidet, was passiert

# Funktionen und Kohesion

-   Eine Funktion erledigt genau eine Aufgabe
-   Aufgabe typischerweise in einem knappen Satz beschreibbar
-   Wenn zu kompliziert, zu viele unterschiedliche Aspekte: Funktion 
    in Teilfunktionen aufteilen!

# Funktionen und Größe

-   Funktion sollte relativ klein sein (Anzahl Code-Zeilen)
-   Faustregel: zwei Bildschirmseiten als Maximum
-   Wenn größer: Funktion 
    in Teilfunktionen aufteilen!

<h1>Overview</h1> 

1. Überblick
1. Flexible Funktionsparameter und Rückgabewerte
1. Funktionen rufen Funktionen auf
1. Indirekte Rekursion
1. Direkte Rekursion
1. Funktionen und Software-Entwurf
1. **<font color="red">Zusammenfassung</font>**

# Zusammenfassung

-   Funktionen können Parameter in flexibler Form entgegen nehmen:
    default-Werte, variable Parameteranzahl, Aufruf mit Parameter-Namen
-   Funktionen können beliebige Typen zurückgeben, insbesondere Tuple
    -   Nützlich um mehrere unterschiedliche Werte als Resultat zu
        verwenden
-   Jeder **Funktionsaufruf** hat einen eigenen Namensraum, in dem Namen
    lokal existieren
    -   Entsteht bei Funktionsaufruf, zerstört bei Funktionsende
-   Der **Scope** eines Namens ist lexikalisch durch den Ort der Zuweisung
    an den Namen bestimmt
    -   Bei Nutzung eines Namens wird zunächst der lokale, dann der
        globale Scope besucht
-   Funktionen können direkte oder indirekte Rekursion nutzen
    -   Jeweils mit eigenem Namensraum

# Python-Keywords: Liste bis jetzt

-   Bis jetzt:
    -   `True`, `False`, `and`, `or`,  `def`,  `return`, `None`
    -   `in`
    -   `if`, `else`, `pass`, `elif`, `while`, `for`, `break`, `continue`, `assert`
-   Neu:
    -   `global`

  